In [95]:
import glob
import collections
import pickle
import numpy as np
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model

def train_network():
    """ Train a Neural Network to generate music """
    notes_by_instrument = get_notes()

    # get amount of pitch names
    n_vocab = 0
    
    for instrument_key, notes in notes_by_instrument.items():
        n_vocab += len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes(source_path="midi_data/multi/*.mid", dest_path="notes_data/all_notes"): # need to modify the path of the directory 
    """ Get all the notes and chords from the midi files from  """
    notes_by_instrument = collections.defaultdict(list)
    for file in glob.glob(source_path):
        # converting .mid file to stream object
        midi = converter.parse(file)
        print("Parsing %s" % file)
        parts = None
        notes_to_parse = {}
        
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            print('parts raw = ', parts)
            for part in parts.parts:
                if str(part.getInstrument()):
                    print(part.getInstrument())
                    notes_to_parse[str(part.getInstrument())] = part.recurse()
                else:
                    pass
        for instrument_key, note_to_parse_by_instrument in notes_to_parse.items():
            for element in note_to_parse_by_instrument: 
                if isinstance(element, note.Note):
                    # if element is a note, extract pitch
                    notes_by_instrument[instrument_key].append(str(element.pitch))
                elif(isinstance(element, chord.Chord)):
                    # if element is a chord, append the normal form of the 
                    # chord (a list of integers) to the list of notes. 
                    notes_by_instrument[instrument_key].append('.'.join(str(n) for n in element.normalOrder))
                
    with open(dest_path, 'wb') as filepath:
        pickle.dump(notes_by_instrument, filepath)
    
    
    return notes_by_instrument

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    
    if np.any(network_output):
        network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history = model.fit(network_input,network_output, validation_split=0.20, epochs=1, batch_size=128, callbacks=callbacks_list)
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    



In [97]:
get_notes()

Parsing midi_data/multi/CameronLeeSimpson-.mid
parts raw =  <music21.stream.Score 0x1411afb00>
Piano
Parsing midi_data/multi/Cameron_Lee_Simpson_-_Watermelon_Rag.mid
parts raw =  <music21.stream.Score 0x119bedac8>
Piano
Parsing midi_data/multi/AndreandSchwandt-aLittleDream.mid
parts raw =  <music21.stream.Score 0x14a034fd0>
Violin
Piano
Parsing midi_data/multi/Moab_Berckmans_de_Oliveira_-_MBOM06 (3).mid
parts raw =  <music21.stream.Score 0x141cbf080>
Harpsichord
Piano
Parsing midi_data/multi/Berckman-Berckman.mid
parts raw =  <music21.stream.Score 0x14430a550>
Piano


defaultdict(list,
            {'Piano': ['3.5.9',
              '4.6.10',
              '3.5.9',
              '4.6.10',
              '3.5.9',
              '4.6.10',
              '3.5.9',
              '9.0.3.5',
              '5',
              '7',
              '9',
              '10',
              '10.2.5',
              '10.2.5',
              '5',
              '2.5.7',
              '10.2.5',
              '10',
              '10.2.5',
              '10.2.5',
              '5',
              '2.5.7',
              '10.2.5',
              '6',
              '6.10.1',
              '6.10.1',
              '1',
              '10.1.3',
              '6.10.1',
              '6',
              '6.10.1',
              '6.10.1',
              '1',
              '10.1.3',
              '6.10.1',
              '10',
              '2.5.7.10',
              '10.2.5',
              '5',
              '2.5.7.9.10',
              '10.2.5',
              '10',
              '2.5.7.10',
    

In [4]:

train_network()


4/4 [==============================] - 80s 20s/step - loss: 4.8511 - val_loss: 4.5951
dict_keys(['loss', 'val_loss'])


NameError: name 'plt' is not defined

In [44]:
#  %run predict_v2.py 